In [1]:
import redis
import torndb
import json
import random


def date_handler(obj):
    if hasattr(obj, 'isoformat'):
        return obj.isoformat()
    else:
        raise TypeError

pool = redis.ConnectionPool(host='192.168.55.121', port=6379, password='foobar', decode_responses=True)


# pool = redis.ConnectionPool(host='123.206.254.42', port=6379, password='#q0AakIO8u*MIG*e', decode_responses=True)
db = torndb.Connection("123.206.254.42:3306", "jinshenxiangmu",
                       user="tong", password="DeepDarkM")
# torndb.pytorndb.py
r = redis.Redis(connection_pool=pool)




In [ ]:
# 设置项目详情
sql = 'SELECT userID, project.projectID,projectSerial,projectName,projectBudgetSum,projectStartTime,projectEndTime,projectStatus,projectComment,projectDelete,budgetID,username,departmentName,projectDepartmentID  FROM project left JOIN department ON projectDepartmentID=departmentID left JOIN `user` ON projectChargerID=userID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)x
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Project:ID:"+str(i.get("projectID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

In [ ]:
# 设置项目列表
# r.zadd("Project:All:Default",)
sql = 'SELECT project.projectID from project'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = str(i.get("projectID"))
    key = "Project:All:Default"
#     print (key+value)
    r.zadd(key, i.get("projectID"), value)
pipe.execute()

In [ ]:
# 测试获取项目列表
prolist = r.zrange("Project:All:Default", 1, 2)
result=[]
keys=[]
for i in prolist:
    key = "Project:ID:"+i
    result.append(json.loads(r.get(key)))
print(result)

In [ ]:
r.get("Project:ID:1")

In [ ]:
# 所有用户列表
sql = 'SELECT userID from user'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = str(i.get("userID"))
    key = "User:All:Default"
#     print (key+value)
    r.zadd(key,value, value)
pipe.execute()

In [ ]:
# 用户的项目列表
userList = r.zrange("User:All:Default", 0 ,-1)
sql = 'SELECT project.projectSerial,project.projectID from project where projectChargerID = %s'
pipe = r.pipeline(transaction=True)
for j in userList:
    result = db.query(sql, j)
    for i in result:
        value = str(i.get("projectSerial"))
        key = "ProjectSerial:User:"+j
        pipe.zadd(key, value,i.get("projectID") )
pipe.execute()

In [ ]:
# 用户的项目下的合同列表
userList = r.zrange("User:All:Default", 0 ,-1)
sql = 'SELECT contractSerial,contractID FROM contract WHERE projectID= %s'
pipe = r.pipeline(transaction=True)
for j in userList:
    for i in r.zrange("Project:User:"+j,0,-1):
        for k in db.query(sql,i):
            value = str(k.get("contractSerial"))
            key1 = "ContractSerial:User:"+j+":Project:"+i
            key2 = "ContracSerialt:User:"+j
            pipe.zadd(key1, value, k.get("contractID"), )
            pipe.zadd(key2, value, k.get("contractID"))


pipe.execute()
        


In [ ]:
# 设置合同详情
sql = 'SELECT contractID,contractSerial,contractName,contractPartner,contractSignedTime,contractMoney,contractComment,projectSerial,contractDelete,contractOwner,contractContent FROM contract INNER JOIN project ON project.projectID = contract.projectID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Contract:ID:"+str(i.get("contractID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

In [ ]:
# 设置文档详情
sql = 'SELECT documentID, documentSerial, documentName, documentUploadTime, documentOwner , documentComment, documentUrl, documentType, documentDelete, projectSerial , contractSerial FROM document INNER JOIN project ON document.`projectID` = project.`projectID` INNER JOIN contract ON document.contractID = contract.contractID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Document:ID:"+str(i.get("documentID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

In [ ]:
# 设置文档详情
sql = 'SELECT documentID, documentSerial, documentName, documentUploadTime, documentOwner , documentComment, documentUrl, documentType, documentDelete, projectSerial , contractSerial FROM document INNER JOIN project ON document.`projectID` = project.`projectID` INNER JOIN contract ON document.contractID = contract.contractID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Document:ID:"+str(i.get("documentID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

In [ ]:
# 设置资产详情
sql = 'SELECT assetID, assetSerial, assetName, assetSpec, assetModel , assetAmount, assetMoney, assetDate, assetOwner, assetComment , assetStatus, assetDelete, projectSerial, assetDepID FROM asset INNER JOIN project ON asset.projectID = project.projectID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Asset:ID:"+str(i.get("assetID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

In [ ]:
# 设置财务详情
sql = 'SELECT financeID, financeSerials, financeDate, financeName, financeMoney , financeStatus, financeDelete, borrowName, lendName, buyOrgFormInfo , buyMethodInfo, buyTypeInfo, payMethodInfo, projectSerial, contractSerial FROM finance INNER JOIN ( SELECT accountantID, accountantName AS borrowName FROM accountant ) borrow ON financeBorrowerID = borrow.accountantID INNER JOIN ( SELECT accountantID, accountantName AS lendName FROM accountant ) lend ON financeLenderID = lend.accountantID INNER JOIN buyorgform ON financeBuyOrgID = buyOrgFormID INNER JOIN buymethod ON financeBuyMethodID = buyMethodID INNER JOIN buytype ON financeTypeID = buyTypeID INNER JOIN paymethod ON finance.payMethodID = paymethod.`payMethodID` INNER JOIN project ON finance.`projectID` = project.`projectID` INNER JOIN contract ON finance.`contractID` = contract.`contractID`'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Finance:ID:"+str(i.get("financeID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

In [ ]:
# 设置预算详情
sql = 'SELECT budgetdetail.budgetID,budgetSerial,project.projectBudgetSum,projectContractsSum,projectContractsPayed,projectContractsNotPayed,projectBudgetLeft,project.projectID,projectSerial,projectName,projectStartTime FROM budgetdetail INNER JOIN project ON budgetdetail.projectID=project.projectID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Budget:ID:"+str(i.get("budgetID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

In [ ]:
# 查 选择借方信息 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  借方下拉框
sql2= "SELECT accountantID,accountantName,accountantType,versionID FROM accountant inner join version on accountant.versionID = version.verisionID WHERE accountantType = 0 and versionBuildNum = %s"
resultHtml = '<option value="">请选择贷方信息</option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'


    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["accountantID"] +'">' + j["accountantName"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "BorrowType:list"
   
r.set(key, resultHtml)
    
   

In [ ]:
# 查 贷方下拉框 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  贷方下拉框
sql2= " SELECT accountantID,accountantName,accountantType,versionID FROM accountant inner join version on accountant.versionID = version.verisionID WHERE accountantType = 1 and versionBuildNum = %s"
resultHtml = '<option value="">请选择贷方信息</option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
        
        
 
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["accountantID"] +'">' + j["accountantName"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "LendType:list"
   
r.set(key, resultHtml)

In [ ]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  预算批复下拉框
sql2= " SELECT budgetReplyID,budgetReplyInfo,versionID FROM budgetreply inner join version on budgetreply.versionID = version.verisionID WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["budgetReplyID"] +'">' + j["budgetReplyInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "BudgetReply:list"
   
r.set(key, resultHtml)

In [ ]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  建设内容下拉框
sql2= " SELECT buildContentID,buildInfo,versionID FROM buildcontent inner join version on buildcontent.versionID = version.verisionID WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["buildContentID"] +'">' + j["buildInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "BuildContent:list"
   
r.set(key, resultHtml)

In [2]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  采购方式下拉框
sql2= " SELECT buyMethodID,buyMethodInfo,versionID FROM buymethod inner join version on buymethod.versionID = version.verisionID WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["buyMethodID"] +'">' + j["buyMethodInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "PurchaseWay:list"
   
r.set(key, resultHtml)

<option value=""></option><optgroup label="v4.0"></optgroup><optgroup label="v3.0（已失效）"><option value="3">buymethod3</option></optgroup><optgroup label="v2.0（已失效）"><option value="2">buymethod2</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">buymethod1</option></optgroup>


True

In [ ]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  采购组织方式下拉框
sql2= " SELECT buyOrgFormID,buyOrgFormInfo,versionID FROM buyorgform inner join version on buyorgform.versionID = version.verisionID  WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2,versionBuildNum):
        resultHtml += '<option value="' + j["buyOrgFormID"] +'">' + j["buyOrgFormInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "BuyOrgMethod:list"
   
r.set(key, resultHtml)

In [ ]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  采购类型下拉框
sql2= " SELECT buyTypeID,buyTypeInfo,versionID FROM buytype inner join version on buytype.versionID = version.verisionID  WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["buyTypeID"] +'">' + j["buyTypeInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "PurchaseType:list"
   
r.set(key, resultHtml)

In [ ]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  部门下拉框
sql2= " SELECT departmentID,departmentName,versionID FROM department inner join version on department.versionID = version.verisionID  WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["departmentID"] +'">' + j["departmentName"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "Department:list"
   
r.set(key, resultHtml)

In [ ]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  付款方式下拉框
sql2= " SELECT payMethodID,payMethodInfo,versionID FROM paymethod inner join version on paymethod.versionID = version.verisionID  WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["payMethodID"] +'">' + j["payMethodInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "PayMethod:list"
   
r.set(key, resultHtml)

In [ ]:
# 权限下拉框
sql = 'SELECT permissionID,url,LEVEL,permissionComment FROM permission'
result = db.query(sql)
value = json.dumps(i, default=date_handler)
key = "Level:list"
r.set(key, value)

In [ ]:
# 项目钱数相关缓存

sql ='select projectID from project'
projectList = []
for i in db.query(sql):
    projectList.append(i["projectID"])
sql2 = 'select contractMoney from contract where projectID = %s'

pipe = r.pipeline(transaction=True)
for projectID in projectList:
    contractSum = 0
    for contract in db.query(sql2, projectID):
        contractSum += contract["contractMoney"]
#     print(contractSum, projectID)
    key = 'Project:'+str(projectID)+':Contract:Sum'
    
    pipe.set(key, contractSum)
pipe.execute()

In [ ]:
r.get("Project:3:Contract:Sum")

In [ ]:
# 项目钱数相关缓存

sql ='select contractID, contractMoney from contract'
contractList = []
pipe = r.pipeline(transaction=True)
for i in db.query(sql):
    contractList.append(i["contractID"])
    pipe.set( 'Contract:'+str(i["contractID"])+":AllMoney", i["contractMoney"])
pipe.execute()

# contractMoneyList = []
pipe = r.pipeline(transaction=True)


for i in db.query(sql):
# contractList
    sql2 = 'select financeMoney from finance where contractID = %s and financeStatus = 0' 
    sql3 = 'select financeMoney from finance where contractID = %s and financeStatus = 1' 
    for contractID in contractList:
        financeSum1 = 0
        financeSum2 = 0
        for finance in db.query(sql2, contractID):
            financeSum1 += finance["financeMoney"]
        key = 'Contract:'+str(contractID)+':SumPay'
        pipe.set(key, financeSum1)
        for finance in db.query(sql3, contractID):
            financeSum2 += finance["financeMoney"]
        key = 'Contract:'+str(contractID)+'FinUnPay'
        pipe.set(key, financeSum2)
        key = 'Contract:'+str(contractID)+':Extra'
        financeExtra =  int(r.get( 'Contract:'+str(contractID)+":AllMoney")) - financeSum1 - financeSum2
        pipe.set(key, financeExtra)

    

#     for contract in db.query(sql2, projectID):
#         contractSum += contract["contractMoney"]
# #     print(contractSum, projectID)
#     key = 'Project:'+str(projectID)+':Contract:Sum'
    
#     pipe.set(key, contractSum)
pipe.execute()

In [ ]:
print(r.get('Contract:240:AllMoney'),r.get('Contract:240:SumPay'),r.get('Contract:240:UnPay'),r.get('Contract:240:FinUnPay'),r.get('Contract:240:Extra'))




In [ ]:
# 设置用户信息缓存

sql = 'select * from user '
pipe = r.pipeline(transaction=True)


for i in db.query(sql):
    key = "User:ID:"+str(i["userID"])
    value = json.dumps(i)
    pipe.set(key, value)
pipe.execute()